In [7]:
import pandas as pd
import json
import re
import matplotlib.pyplot as plt

# Load the JSON data from the file
with open(r"C:\Users\ADMIN\Desktop\Data science datasets\apple_specifications.json", 'r', encoding='utf-8') as file:
    apple_data = json.load(file)

with open(r"C:\Users\ADMIN\Desktop\Data science datasets\asus_specifications.json", 'r', encoding='utf-8') as file:
    asus_data = json.load(file)

with open(r"C:\Users\ADMIN\Desktop\Data science datasets\acatel_specifications.json", 'r', encoding='utf-8') as file:
    acatel_data = json.load(file)

with open(r"C:\Users\ADMIN\Desktop\Data science datasets\samsung_specifications.json", 'r', encoding='utf-8') as file:
    samsung_data = json.load(file)

with open(r"C:\Users\ADMIN\Desktop\Data science datasets\xiaomi_specifications.json", 'r', encoding='utf-8') as file:
    xiaomi_data = json.load(file)

with open(r"C:\Users\ADMIN\Desktop\Data science datasets\oppo_specifications.json", 'r', encoding='utf-8') as file:
    oppo_data = json.load(file)

with open(r"C:\Users\ADMIN\Desktop\Data science datasets\huawei_specifications.json", 'r', encoding='utf-8') as file:
    huawei_data = json.load(file)

with open(r"C:\Users\ADMIN\Desktop\Data science datasets\realme_specifications.json", 'r', encoding='utf-8') as file:
    realme_data = json.load(file)

In [ ]:
def convert_price_to_usd(price_str):
    if not price_str or price_str.strip() == '':  
        return 'N/A'  
    
    match = re.search(r'([\$\€\£\₹\¥])?\s*([\d,\.]+)', price_str)
    if match:
        symbol = match.group(1)
        amount = match.group(2).replace(',', '')
    
        exchange_rates = {
    "USD": 1.0,  
    "EUR": 1.1,  
    "GBP": 1.3, 
    "INR": 0.012, 
    "JPY": 0.0088, 
    "Unknown": 1.0  
}
        
        try:
            amount = float(amount) 
        except ValueError:
            return 'N/A'

        if symbol == '$':
            currency = 'USD'
        elif symbol == '€':
            currency = 'EUR'
        elif symbol == '£':
            currency = 'GBP'
        elif symbol == '₹':
            currency = 'INR'
        elif symbol == '¥':
            currency = 'JPY'
        else:
            currency = 'Unknown'
        return amount * exchange_rates.get(currency, 1.0)
    
    return 'N/A'

def create_device_dataframe(data):
    devices = []
    for item in data:
        name = item.get("name", "N/A")

        specs = item.get("specifications", {})

        dimension = specs.get("dimensions", "N/A")
        dimension_mm = re.sub(r"\s*\(.*?\)|\s*mm", "", dimension).strip()
        match = re.match(r"(\d+\.?\d*)\s*x\s*(\d+\.?\d*)\s*x\s*(\d+\.?\d*)", dimension)
        if match:
            dimension_length = match.group(1)
            dimension_width = match.group(2)
            dimension_thickness = match.group(3)
        else:
            dimension_length = "N/A"
            dimension_width = "N/A"
            dimension_thickness = "N/A"

        weight = specs.get("weight", "N/A")
        weight_value = re.search(r"(\d+)", weight) 
        weight = weight_value.group(1) if weight_value else "N/A" 

        battery_type = specs.get("type", "N/A")
        battery_capacity = re.search(r"(\d+)\s*mAh", battery_type) 
        battery_capacity = battery_capacity.group(1) if battery_capacity else "N/A" 
        if "Li-Ion" in battery_type:
            battery_type = 1
        elif "Li-Po" in battery_type:
            battery_type = 0
        else:
            battery_type = 1

        size = specs.get("size", "N/A")
        size_cm2 = re.search(r"(\d+\.?\d*)\s*cm2", size)
        size = size_cm2.group(1) if size_cm2 else "N/A" 

        resolution = specs.get("resolution", "N/A")
        resolution = re.sub(r"\s*pixels.*", "", resolution).strip()
        # Tách giá trị height và width
        match = re.match(r"(\d+)\s*x\s*(\d+)", resolution)
        if match:
            resolution_height = match.group(1) 
            resolution_width = match.group(2)  
        else:
            resolution_height = "N/A"
            resolution_width = "N/A"

        cpu = specs.get("cpu", "N/A")
        cpu = re.sub(r"(core).*", r"\1", cpu).strip()
        if "Hexa-core" or "6-core" in cpu:
                cpu = 6
        elif "Quad-core" or "4-core" in cpu:
                cpu = 4
        elif "Octa-core" or "8-core" in cpu:
                cpu = 8
        elif "Dual-core" or "2-core" in cpu:
                cpu = 2
        else:
                cpu = "N/A"


        internal_storage = specs.get("internal", "N/A")
        match = re.match(r"(\d+)\w+\s+(\d+)\w+\s+RAM", internal_storage)
        if match:
            internal_storage = match.group(1) 
            ram = match.group(2)   
        else:
            internal_storage = "N/A"
            ram = "N/A"

        nfc = specs.get("nfc", "N/A")
        nfc = re.match(r"(Yes|No)", nfc).group(1) if re.match(r"(Yes|No)", nfc) else "N/A"
        if "Yes" in nfc:
             nfc = 1
        else :
             nfc = 0

        price = specs.get("price", "N/A")

        if isinstance(price, str):
            try:
                price = price.encode('utf-8').decode('utf-8')
            except UnicodeEncodeError:
                price = price.encode('ascii', 'ignore').decode('ascii')
        
        if isinstance(price, str):
            price_in_usd = convert_price_to_usd(price)
        else:
            price_in_usd = 'N/A'

        devices.append({
            "Brand": name,
            "Dimension Leight (mm)": dimension_length,
            "Dimension Width (mm)": dimension_width,
            "Dimesion Thickness (mm)": dimension_thickness,
            "Weight (g)": weight,
            "Battery Type" : battery_type,
            "Battery Capacity (mAh)": battery_capacity,
            "Screen Size (cm2)": size,
            "Resolution Height (pixels)":resolution_height,
            "Resolution Width (pixels)":resolution_width,
            "CPU (cores)": cpu,
            "Internal Storage (GB)": internal_storage,
            "RAM (GB)": ram,
            "NFC Support": nfc,
            "Price (USD)": price_in_usd
        })
    
    df = pd.DataFrame(devices)
    df = df.replace("N/A", pd.NA).dropna(how='any').reset_index(drop=True)

    return df


In [9]:
# Create DataFrames for each brand
df_apple = create_device_dataframe(apple_data)
df_asus = create_device_dataframe(asus_data)
df_acatel = create_device_dataframe(acatel_data)
df_samsung = create_device_dataframe(samsung_data)
df_xiaomi = create_device_dataframe(xiaomi_data)
df_oppo = create_device_dataframe(oppo_data)
df_huawei = create_device_dataframe(huawei_data)
df_realme = create_device_dataframe(realme_data)

# Display the DataFrames
print("Apple Specifications:")
print(f"Total products: {len(df_apple)}")
display(df_apple.head(5))

print("\nASUS Specifications:")
print(f"Total products: {len(df_asus)}")
display(df_asus.head(5))

print("\nAcatel Specifications:")
print(f"Total products: {len(df_acatel)}")
display(df_acatel.head(5))

print("\nSamsung Specifications:")
print(f"Total products: {len(df_samsung)}")
display(df_samsung.head(5))

print("\nXiaomi Specifications:")
print(f"Total products: {len(df_xiaomi)}")
display(df_xiaomi.head(5))

print("\nOppo Specifications:")
print(f"Total products: {len(df_oppo)}")
display(df_oppo.head(5))

print("\nHuawei Specifications:")
print(f"Total products: {len(df_huawei)}")
display(df_huawei.head(5))

print("\nRealme Specifications:")
print(f"Total products: {len(df_realme)}")
display(df_realme.head(5))

Apple Specifications:
Total products: 93


,Brand,Dimension Leight (mm),Dimension Width (mm),Dimesion Thickness (mm),Weight (g),Battery Type,Battery Capacity (mAh),Screen Size (cm2),Resolution Height (pixels),Resolution Width (pixels),CPU (cores),Internal Storage (GB),RAM (GB),NFC Support,Price (USD)
0,Apple iPad mini (2024),195.4,134.8,6.3,293,1,5078,203.9,1488,2266,6,128,8,0,600.0
1,Apple iPhone 16 Pro Max,163,77.6,8.3,227,1,4685,115.6,1320,2868,6,256,8,1,1199.99
2,Apple iPhone 16 Pro,149.6,71.5,8.3,199,1,3582,96.4,1206,2622,6,128,8,1,999.99
3,Apple iPhone 16 Plus,160.9,77.8,7.8,199,1,4674,110.2,1290,2796,6,128,8,1,899.0
4,Apple iPhone 16,147.6,71.6,7.8,170,1,3561,91.7,1179,2556,6,128,8,1,829.99



ASUS Specifications:
Total products: 78


,Brand,Dimension Leight (mm),Dimension Width (mm),Dimesion Thickness (mm),Weight (g),Battery Type,Battery Capacity (mAh),Screen Size (cm2),Resolution Height (pixels),Resolution Width (pixels),CPU (cores),Internal Storage (GB),RAM (GB),NFC Support,Price (USD)
0,Asus Zenfone 4 Selfie ZD553KL,155.7,75.9,7.9,144,1,3000,83.4,720,1280,6,64,4,0,250.0
1,Asus Zenfone 4 Max ZC520KL,150.5,73.3,8.8,156,0,4100,74.5,720,1280,6,16,2,0,180.0
2,Asus Zenfone 4 Max Pro ZC554KL,154,76.9,8.9,181,0,5000,83.4,720,1280,6,32,2,0,250.0
3,Asus Zenfone 4 Max Plus ZC554KL,154,76.9,8.9,181,0,5000,83.4,720,1280,6,32,3,0,220.0
4,Asus Zenfone 4 Max ZC554KL,154,76.9,8.9,181,0,5000,83.4,720,1280,6,32,3,0,270.0



Acatel Specifications:
Total products: 99


...

,Brand,Dimension Leight (mm),Dimension Width (mm),Dimesion Thickness (mm),Weight (g),Battery Type,Battery Capacity (mAh),Screen Size (cm2),Resolution Height (pixels),Resolution Width (pixels),CPU (cores),Internal Storage (GB),RAM (GB),NFC Support,Price (USD)
0,Oppo Find X8,157.4,74.3,7.9,193,1,5630,105.6,1256,2760,6,256,12,1,550.0
1,Oppo Pad 3 Pro,268.7,195.1,6.5,586,1,9510,445.2,2120,3000,6,256,8,0,430.0
2,Oppo K12 Plus,162.5,75.3,8.4,192,1,6400,108.0,1080,2412,6,256,8,1,250.0
3,Oppo A80,165.8,76.1,7.7,186,1,5100,107.2,720,1604,6,128,8,1,200.0
4,Oppo F27,163,75.8,7.7,187,1,5000,107.4,1080,2400,6,128,8,1,233.28



Huawei Specifications:
Total products: 93


,Brand,Dimension Leight (mm),Dimension Width (mm),Dimesion Thickness (mm),Weight (g),Battery Type,Battery Capacity (mAh),Screen Size (cm2),Resolution Height (pixels),Resolution Width (pixels),CPU (cores),Internal Storage (GB),RAM (GB),NFC Support,Price (USD)
0,Huawei MatePad 12 X,270,183,5.9,555,0,10100,426.4,2800,1840,6,256,8,0,850.0
1,Huawei MatePad Air (2024),270,183,5.9,555,0,10100,426.4,2800,1840,6,256,8,0,370.0
2,Huawei MatePad Pro 12.2 (2024),271.3,182.5,5.5,508,0,10100,440.7,2800,1840,6,256,12,0,570.0
3,Huawei Pura 70 Ultra,162.6,75.1,8.4,226,0,5200,109.0,1260,2844,6,512,16,1,1292.49
4,Huawei Pura 70 Pro,162.6,75.1,8.4,220,0,5050,109.0,1260,2844,6,256,12,1,1300.0



Realme Specifications:
Total products: 128


,Brand,Dimension Leight (mm),Dimension Width (mm),Dimesion Thickness (mm),Weight (g),Battery Type,Battery Capacity (mAh),Screen Size (cm2),Resolution Height (pixels),Resolution Width (pixels),CPU (cores),Internal Storage (GB),RAM (GB),NFC Support,Price (USD)
0,Realme GT7 Pro,162.5,76.9,8.6,222,1,6500,111.7,1264,2780,6,256,12,1,470.0
1,Realme P1 Speed,161.7,74.7,7.6,185,1,5000,107.4,1080,2400,6,128,8,0,202.176
2,Realme P2 Pro,161.3,73.9,8.2,180,1,5200,108.0,1080,2412,6,128,8,0,307.2
3,Realme Pad 2 Lite,258,169.5,8.3,525,1,8300,347.7,1200,1920,6,128,4,0,160.0
4,Realme Narzo 70 Turbo,161.7,74.7,7.6,185,1,5000,107.4,1080,2400,6,128,6,0,203.976


In [10]:
# Combine all DataFrames into one
df_all = pd.concat([df_apple, df_asus, df_acatel, df_samsung, df_xiaomi, df_oppo, df_huawei, df_realme], ignore_index=True)
df_all.head(5)

# Export the combined DataFrame to a CSV and JSON file
df_all.to_csv(r"C:\Users\ADMIN\Desktop\combined_device_data_2.csv", index=False, encoding='utf-8')
# Corrected line for exporting to JSON
df_all.to_json(r"C:\Users\ADMIN\Desktop\combined_device_data_2.json", orient='records', force_ascii=False, indent=4)

In [11]:
# Perform EDA on each DataFrame
def perform_eda(df, brand_name):
    print(f"\nEDA for {brand_name}:")
    print(df.describe(include='all'))
    
    # Check for missing values
    print(f"\nMissing Values for {brand_name}:")
    print(df.isnull().sum())
    
    # Plot distribution of numerical columns
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    if not numeric_cols.empty:
        df[numeric_cols].hist(figsize=(15, 10), bins=15)
        plt.suptitle(f"Distribution of Numerical Features for {brand_name}")
        plt.show()

In [13]:
# Combine all DataFrames into one
df_all = pd.concat([df_apple, df_asus, df_acatel, df_samsung, df_xiaomi, df_oppo, df_huawei, df_realme], ignore_index=True)